# SnP EDA + Data Preparation

## preprocessing SnP PPI data

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
os.chdir('/content/drive/MyDrive/Programmable Biology Group/Srikar/Code/proteins/flamingo-ppi-gen/data_dump/per-residue-dataset/')

In [3]:
!ls

protT5_embeddings.pkl  testing_dataset.csv   validation_dataset.csv
protT5_tokens.pkl      training_dataset.csv


In [4]:
import pandas as pd

In [5]:
test_snp = pd.read_csv('testing_dataset.csv')
train_snp = pd.read_csv('training_dataset.csv')
val_snp = pd.read_csv('validation_dataset.csv')

In [6]:
import pandas as pd
import re

def preprocess_snp_data(file_path):
    # Read the dataset
    snp_df = pd.read_csv(file_path)

    # Function to transform energy scores
    def transform_energy_scores(energy_scores):
        transformed_scores = []
        for score in energy_scores:
            # Replace sequences of spaces/newlines with a comma
            score = re.sub(r'[\s\n]+', ',', score)
            # Remove a comma after an opening square bracket
            score = re.sub(r'\[\s*,', '[', score)
            # Remove leading commas/whitespace
            score = re.sub(r'^[\s,]+', '', score)
            transformed_scores.append(score)
        return transformed_scores

    # Apply transformations
    snp_df['energy_scores'] = transform_energy_scores(snp_df['energy_scores'])
    snp_df['energy_scores_lengths'] = snp_df['energy_scores'].apply(
        lambda x: x.count(',') + 1 - (1 if x.startswith(',') else 0)
    )

    # Calculate lengths for other columns
    snp_df['peptide_source_RCSB_lengths'] = snp_df['peptide_source_RCSB'].apply(len)
    snp_df['protein_RCSB_lengths'] = snp_df['protein_RCSB'].apply(len)
    snp_df['protein_derived_seq_length'] = snp_df['protein_derived_sequence'].apply(len)
    snp_df['peptide_derived_seq_length'] = snp_df['peptide_derived_sequence'].apply(len)

    # Calculate matching lengths count (optional, depending on your needs)
    snp_df['matching_lengths_count'] = (snp_df['energy_scores_lengths'] == snp_df['peptide_derived_seq_length']).sum()

    return snp_df

# Applying the preprocessing pipeline to each dataset
test_snp = preprocess_snp_data('testing_dataset.csv')
train_snp = preprocess_snp_data('training_dataset.csv')
val_snp = preprocess_snp_data('validation_dataset.csv')


In [7]:
train_snp['protein_derived_sequence'][0]

'VWLANPERYGQMQYRYCGKSGLRLPALSLGLWHNFGHVNALESQRAILRKAFDLGITHFDLANNYGPPPGSAEENFGRLLREDFAAYRDELIISTKAGYDMWPGPYGSGGSRKYLLASLDQSLKRMGLEYVDIFYSHRVDENTPMEETASALAHAVQSGKALYVGISSYSPERTQKMVELLREWKIPLLIHQPSYNLLNRWVDKSGLLDTLQNNGVGCIAFTPLAQGLLTGKYLTEANLNSLRLLNEMAQQRGQSMAQMALSWLLKDDRVTSVLIGASRAEQLEENVQALNNLTFSTKELAQIDQHIADGELN'

## dataset + dataloaders (includes 1-hot encoding of energy scores)

In [ ]:
def one_hot_encode_energy_scores(scores):
    # Assuming 'scores' is a list of energy score values
    return [1 if score <= -1 else 0 for score in scores]

In [ ]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
import re

class ProteinInteractionDataset(Dataset):
    def __init__(self, dataframe, protT5_embeddings, tokenizer):
        self.dataframe = dataframe
        self.protT5_embeddings = protT5_embeddings
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        peptide_seq = self.dataframe.iloc[idx]['peptide_derived_sequence']
        protein_seq = self.dataframe.iloc[idx]['protein_derived_sequence']
        energy_scores = self.dataframe.iloc[idx]['energy_scores']

        # Tokenize the protein_seq
        tokenized_protein_seq = self.tokenizer.encode(protein_seq, add_special_tokens=True, return_tensors="pt").squeeze()

        # Use regular expression to split the energy_scores string
        energy_scores = re.findall(r'-?\d+\.?\d*(?:e[-+]?\d+)?', energy_scores)

        # Convert the split strings to floats
        energy_scores = [float(score) for score in energy_scores]

        # One-hot encode the energy scores
        encoded_scores = one_hot_encode_energy_scores(energy_scores)

        peptide_embedding = self.protT5_embeddings[peptide_seq]
        protein_embedding = self.protT5_embeddings[protein_seq]

        return peptide_embedding, protein_embedding, torch.tensor(encoded_scores), tokenized_protein_seq


In [8]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
import re
!pip install sentencepiece
import sentencepiece
import torch
from torch import nn
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.utils.data import DataLoader, Dataset

In [9]:

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
# model = model.half() if device.type == 'cuda' else model.full()


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5EncoderModel(
  (shared): Embedding(128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=4096, bias=False)
              (k): Linear(in_features=1024, out_features=4096, bias=False)
              (v): Linear(in_features=1024, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=16384, bias=False)
              (wo): Linear(in_features=16384, out_features=1024, bias=False)
              (dropout): Dropo

In [23]:

from tqdm import tqdm

def generate_protT5_embeddings_tokens(sequences):
    embeddings_dict = {}
    tokens_dict = {}
    counter=0
    # Wrap the sequence iteration with tqdm for a progress bar
    for sequence in tqdm(sequences, desc="Generating..."):
        # Process sequence
        print(len(sequence))
        processed_seq = " ".join(list(re.sub(r"[UZOB]", "X", sequence)))
        print(len(processed_seq))
        # Tokenize and encode
        ids = tokenizer(processed_seq, add_special_tokens=True, return_tensors="pt",padding='longest')
        input_ids = ids['input_ids'].to(device)
        tokens_dict[sequence] = input_ids.squeeze().tolist()
        #print(tokens_dict[sequence])
        attention_mask = ids['attention_mask'].to(device)
        # Generate embeddings
        with torch.no_grad():
            embedding_repr = model(input_ids=input_ids, attention_mask=attention_mask)
        embeddings_dict[sequence] = embedding_repr.last_hidden_state[0,:len(processed_seq)]
        counter = counter+1
        print(embeddings_dict[sequence].shape)
        #print(embeddings_dict[sequence])
    return embeddings_dict,tokens_dict


In [117]:
# train_snp=train_snp.sample(n=5000)
# test_snp = test_snp.sample(n=2000)
# val_snp = test_snp.sample(2000)

In [24]:
from torch.utils.data import DataLoader

# Generate ProtT5 embeddings
# Combine unique sequences from peptide and protein derived sequences
unique_seqs = pd.concat([train_snp['peptide_derived_sequence'], train_snp['protein_derived_sequence'],
                         test_snp['peptide_derived_sequence'], test_snp['protein_derived_sequence'],
                         val_snp['peptide_derived_sequence'], val_snp['protein_derived_sequence']]).unique()

In [25]:
len(unique_seqs)

9665

In [26]:

proT5_embeddings,protT5_tokens = generate_protT5_embeddings_tokens(unique_seqs)


Generating...:   0%|          | 2/9665 [00:00<08:07, 19.81it/s]

315
629
torch.Size([316, 1024])
510
1019
torch.Size([511, 1024])
116
231


Generating...:   0%|          | 6/9665 [00:00<10:25, 15.44it/s]

torch.Size([117, 1024])
217
433
torch.Size([218, 1024])
404
807
torch.Size([405, 1024])
369
737
torch.Size([370, 1024])
87
173


Generating...:   0%|          | 12/9665 [00:00<08:02, 19.99it/s]

...

torch.Size([368, 1024])
278
555
torch.Size([279, 1024])
101
201
torch.Size([102, 1024])
89
177
torch.Size([90, 1024])
139
277
torch.Size([140, 1024])
77
153


Generating...: 100%|█████████▉| 9658/9665 [08:39<00:00, 23.63it/s]

torch.Size([78, 1024])
259
517
torch.Size([260, 1024])
323
645
torch.Size([324, 1024])
130
259
torch.Size([131, 1024])
172
343
torch.Size([173, 1024])
379
757


Generating...: 100%|██████████| 9665/9665 [08:40<00:00, 18.58it/s]

torch.Size([380, 1024])
248
495
torch.Size([249, 1024])
79
157
torch.Size([80, 1024])
307
613
torch.Size([308, 1024])
266
531
torch.Size([267, 1024])


In [119]:
# os.mkdir('/content/drive/MyDrive/Programmable Biology Group/Srikar/Code/proteins/flamingo-ppi-gen/data_dump/per-residue-dataset/')

In [29]:
os.chdir('/content/drive/MyDrive/Programmable Biology Group/Srikar/Code/proteins/flamingo-ppi-gen/data_dump/per-residue-dataset/')

In [30]:
import pickle

In [ ]:
# file_path = 'protT5_embeddings.pkl'
# with open(file_path, 'rb') as file:
#     protT5_embeddings = pickle.load(file)

In [ ]:

# # Create datasets with tokenizer
# train_dataset = ProteinInteractionDataset(train_snp, protT5_embeddings, tokenizer)
# test_dataset = ProteinInteractionDataset(test_snp, protT5_embeddings, tokenizer)
# val_dataset = ProteinInteractionDataset(val_snp, protT5_embeddings, tokenizer)

# # Create DataLoaders
# train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=2)
# val_dataloader = DataLoader(val_dataset, batch_size=2)




## save as .pkl

In [31]:
import pickle

# protT5_embeddings is a dictionary of tensors
with open('protT5_embeddings.pkl', 'wb') as f:
    pickle.dump(proT5_embeddings, f)


In [32]:
len(proT5_embeddings)

9665

In [33]:
import pickle

# token dict
with open('protT5_tokens.pkl', 'wb') as f:
    pickle.dump(protT5_tokens, f)


In [34]:
len(protT5_tokens)

9665

In [123]:
train_snp.to_csv('training_dataset.csv')
test_snp.to_csv('test_dataset.csv')
val_snp.to_csv('val_dataset.csv')

In [ ]:
type(protT5_tokens)

dict

In [ ]:
len('MKLLLFGYGNVGKAFRKLLHEKRSPELNDVIIGGIVTRRGIMLQDKEDFTPDLEGDVFKAFEKIKPDIIVDVSSANYNNGEPSLSLYKEAIKDGVNIITTNKAPLALAFNEIFSLARSKGVKIGFQGTVMSGTPSINLYRVLPGSRVIKIRGILNGTTNFILFEEALKEAQRRGYAEEDPTLDINGFDAAAKITILANFMIGNSVTIKDVKFEGIIKLIAYADEKEVWVKPLPISQDDPLYNVDGVENALEITTDIQSILIRGPGAGPVNAAYGALSDLILLKRDCL')

287

In [ ]:
# protT5_tokens['MKLLLFGYGNVGKAFRKLLHEKRSPELNDVIIGGIVTRRGIMLQDKEDFTPDLEGDVFKAFEKIKPDIIVDVSSANYNNGEPSLSLYKEAIKDGVNIITTNKAPLALAFNEIFSLARSKGVKIGFQGTVMSGTPSINLYRVLPGSRVIKIRGILNGTTNFILFEEALKEAQRRGYAEEDPTLDINGFDAAAKITILANFMIGNSVTIKDVKFEGIIKLIAYADEKEVWVKPLPISQDDPLYNVDGVENALEITTDIQSILIRGPGAGPVNAAYGALSDLILLKRDCL']

In [ ]:
# 19 = M

In [ ]:
# Save the datasets
with open('train_dataset.pkl', 'wb') as f:
    pickle.dump(train_dataset, f)

with open('test_dataset.pkl', 'wb') as f:
    pickle.dump(test_dataset, f)

with open('val_dataset.pkl', 'wb') as f:
    pickle.dump(val_dataset, f)

# Reload Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
os.chdir('/content/drive/MyDrive/Programmable Biology Group/Srikar/Code/proteins/flamingo-ppi-gen/data_dump/per-residue-dataset/')

In [3]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
import re
!pip install sentencepiece
import sentencepiece
import torch
from torch import nn
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModel, AutoTokenizer

In [4]:
def one_hot_encode_energy_scores(scores):
    # Assuming 'scores' is a list of energy score values
    return [1 if score <= -1 else 0 for score in scores]

In [5]:
from torch.utils.data import Dataset
import pickle

In [6]:
# Load the protT5_tokens dictionary
with open('protT5_tokens.pkl', 'rb') as file:
    protT5_tokens = pickle.load(file)

In [7]:
len(protT5_tokens)

9665

In [8]:
len(protT5_tokens)

9665

In [9]:
# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
model = model.half() if device.type == 'cuda' else model.full()

from tqdm import tqdm

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [10]:
import re
import torch
import pickle
from torch.utils.data import Dataset

class ProteinInteractionDataset(Dataset):
    def __init__(self, dataframe, protT5_embeddings):
        self.dataframe = dataframe
        self.protT5_embeddings = protT5_embeddings
        # Load the protT5_tokens dictionary
        self.protT5_tokens = protT5_tokens

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        peptide_seq = self.dataframe.iloc[idx]['peptide_derived_sequence']
        protein_seq = self.dataframe.iloc[idx]['protein_derived_sequence']
        energy_scores = self.dataframe.iloc[idx]['energy_scores']

        # Fetch the tokenized sequence from the dictionary
        tokenized_peptide_seq = self.protT5_tokens[peptide_seq]

        # Use regular expression to split the energy_scores string
        energy_scores = re.findall(r'-?\d+\.?\d*(?:e[-+]?\d+)?', energy_scores)

        # Convert the split strings to floats
        energy_scores = [float(score) for score in energy_scores]

        # One-hot encode the energy scores (assuming this function is defined elsewhere)
        encoded_scores = one_hot_encode_energy_scores(energy_scores)

        peptide_embedding = self.protT5_embeddings[peptide_seq]
        protein_embedding = self.protT5_embeddings[protein_seq]

        # Convert tokenized_protein_seq to a tensor
        tokenized_peptide_seq_tensor = torch.tensor(tokenized_peptide_seq, dtype=torch.float16)

        return peptide_embedding, protein_embedding, torch.tensor(encoded_scores), tokenized_peptide_seq_tensor


In [44]:
import torch
from torch.nn.functional import pad

def collate_fn(batch):
    max_length_embeddings = max(max(prot1_emb.size(0), prot2_emb.size(0)) for prot1_emb, prot2_emb, _, _ in batch)
    max_length_tokenized = max(tok_seq.size(0) for _, _, _, tok_seq in batch)

    prot1_embeddings_padded = []
    prot2_embeddings_padded = []
    one_hot_scores_padded_list = []
    tokenized_seqs_padded = []

    for prot1_emb, prot2_emb, one_hot_scores, tok_seq in batch:
        # Pad only the sequence length dimension
        prot1_emb_padded = pad(prot1_emb, (0, 0, 0, max_length_embeddings - prot1_emb.size(0)), "constant", 0)
        prot2_emb_padded = pad(prot2_emb, (0, 0, 0, max_length_embeddings - prot2_emb.size(0)), "constant", 0)

        if one_hot_scores.nelement() != 0:
            one_hot_scores_padded = pad(one_hot_scores, (0, max_length_embeddings - one_hot_scores.size(0)), "constant", 0)
        else:
            one_hot_scores_padded = one_hot_scores

        tok_seq_padded = pad(tok_seq, (0, max_length_tokenized - tok_seq.size(0)), "constant", 0)

        prot1_embeddings_padded.append(prot1_emb_padded)
        prot2_embeddings_padded.append(prot2_emb_padded)
        one_hot_scores_padded_list.append(one_hot_scores_padded)
        tokenized_seqs_padded.append(tok_seq_padded)

    return torch.stack(prot1_embeddings_padded), torch.stack(prot2_embeddings_padded), torch.stack(one_hot_scores_padded_list), torch.stack(tokenized_seqs_padded)


In [12]:
with open('protT5_embeddings.pkl', 'rb') as file:
    protT5_embeddings = pickle.load(file)

In [13]:
len(protT5_embeddings)

9665

## preprocessing SnP PPI data

In [14]:
os.chdir('/content/drive/MyDrive/Programmable Biology Group/Srikar/Code/proteins/flamingo-ppi-gen/data_dump/per-residue-dataset/')

In [15]:
!ls

protT5_embeddings.pkl  testing_dataset.csv   validation_dataset.csv
protT5_tokens.pkl      training_dataset.csv


In [16]:
import pandas as pd

In [17]:
test_snp = pd.read_csv('testing_dataset.csv')
train_snp = pd.read_csv('training_dataset.csv')
val_snp = pd.read_csv('validation_dataset.csv')

In [18]:
import pandas as pd
import re

def preprocess_snp_data(file_path):
    # Read the dataset
    snp_df = pd.read_csv(file_path)

    # Function to transform energy scores
    def transform_energy_scores(energy_scores):
        transformed_scores = []
        for score in energy_scores:
            # Replace sequences of spaces/newlines with a comma
            score = re.sub(r'[\s\n]+', ',', score)
            # Remove a comma after an opening square bracket
            score = re.sub(r'\[\s*,', '[', score)
            # Remove leading commas/whitespace
            score = re.sub(r'^[\s,]+', '', score)
            transformed_scores.append(score)
        return transformed_scores

    # Apply transformations
    snp_df['energy_scores'] = transform_energy_scores(snp_df['energy_scores'])
    snp_df['energy_scores_lengths'] = snp_df['energy_scores'].apply(
        lambda x: x.count(',') + 1 - (1 if x.startswith(',') else 0)
    )

    # Calculate lengths for other columns
    snp_df['peptide_source_RCSB_lengths'] = snp_df['peptide_source_RCSB'].apply(len)
    snp_df['protein_RCSB_lengths'] = snp_df['protein_RCSB'].apply(len)
    snp_df['protein_derived_seq_length'] = snp_df['protein_derived_sequence'].apply(len)
    snp_df['peptide_derived_seq_length'] = snp_df['peptide_derived_sequence'].apply(len)

    # Calculate matching lengths count (optional, depending on your needs)
    snp_df['matching_lengths_count'] = (snp_df['energy_scores_lengths'] == snp_df['peptide_derived_seq_length']).sum()

    return snp_df

# Applying the preprocessing pipeline to each dataset
test_snp = preprocess_snp_data('testing_dataset.csv')
train_snp = preprocess_snp_data('training_dataset.csv')
val_snp = preprocess_snp_data('validation_dataset.csv')


In [19]:
train_snp['protein_derived_sequence'][0]

'VWLANPERYGQMQYRYCGKSGLRLPALSLGLWHNFGHVNALESQRAILRKAFDLGITHFDLANNYGPPPGSAEENFGRLLREDFAAYRDELIISTKAGYDMWPGPYGSGGSRKYLLASLDQSLKRMGLEYVDIFYSHRVDENTPMEETASALAHAVQSGKALYVGISSYSPERTQKMVELLREWKIPLLIHQPSYNLLNRWVDKSGLLDTLQNNGVGCIAFTPLAQGLLTGKYLTEANLNSLRLLNEMAQQRGQSMAQMALSWLLKDDRVTSVLIGASRAEQLEENVQALNNLTFSTKELAQIDQHIADGELN'

## create the *datasets*

In [45]:
len(protT5_embeddings)

9665

In [46]:
# Create datasets with tokenizer
train_dataset = ProteinInteractionDataset(train_snp, protT5_embeddings)
test_dataset = ProteinInteractionDataset(test_snp, protT5_embeddings)
val_dataset = ProteinInteractionDataset(val_snp, protT5_embeddings)


In [47]:
train_dataset[0]

(tensor([[-0.0241, -0.4068,  0.0734,  ...,  0.2921, -0.2068,  0.0425],
         [-0.0631,  0.0105, -0.1680,  ..., -0.0313, -0.0266,  0.0162],
         [ 0.0673, -0.3383,  0.1790,  ...,  0.1158, -0.0744,  0.0105],
         ...,
         [-0.1477, -0.0070,  0.0498,  ..., -0.2186, -0.1959, -0.0729],
         [-0.2477,  0.0410,  0.1388,  ...,  0.1008, -0.0460, -0.1895],
         [ 0.0312, -0.0547,  0.0142,  ...,  0.0355,  0.0347, -0.0288]],
        device='cuda:0'),
 tensor([[-0.0374, -0.4055,  0.0833,  ...,  0.2913, -0.2128,  0.0282],
         [-0.0690,  0.0085, -0.1689,  ..., -0.0293, -0.0309,  0.0147],
         [ 0.0653, -0.3421,  0.1786,  ...,  0.1258, -0.0753,  0.0121],
         ...,
         [-0.1740,  0.0113, -0.0044,  ...,  0.0643, -0.0128,  0.0029],
         [-0.2512, -0.0716,  0.2274,  ..., -0.0650, -0.0475, -0.1648],
         [ 0.0293, -0.0548,  0.0157,  ...,  0.0341,  0.0372, -0.0240]],
        device='cuda:0'),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [48]:
os.chdir('/content/drive/MyDrive/Programmable Biology Group/Srikar/Code/proteins/flamingo-ppi-gen/data_dump/flamingo-26-data/')

In [49]:
from torch.utils.data import DataLoader

train_batch_size = 2
test_batch_size = 2
val_batch_size = 2

# Create the DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=val_batch_size, collate_fn=collate_fn)


In [50]:
len(train_dataloader),len(test_dataloader),len(val_dataloader)

(2500, 1000, 1000)

In [51]:
train_dataset[0]

(tensor([[-0.0241, -0.4068,  0.0734,  ...,  0.2921, -0.2068,  0.0425],
         [-0.0631,  0.0105, -0.1680,  ..., -0.0313, -0.0266,  0.0162],
         [ 0.0673, -0.3383,  0.1790,  ...,  0.1158, -0.0744,  0.0105],
         ...,
         [-0.1477, -0.0070,  0.0498,  ..., -0.2186, -0.1959, -0.0729],
         [-0.2477,  0.0410,  0.1388,  ...,  0.1008, -0.0460, -0.1895],
         [ 0.0312, -0.0547,  0.0142,  ...,  0.0355,  0.0347, -0.0288]],
        device='cuda:0'),
 tensor([[-0.0374, -0.4055,  0.0833,  ...,  0.2913, -0.2128,  0.0282],
         [-0.0690,  0.0085, -0.1689,  ..., -0.0293, -0.0309,  0.0147],
         [ 0.0653, -0.3421,  0.1786,  ...,  0.1258, -0.0753,  0.0121],
         ...,
         [-0.1740,  0.0113, -0.0044,  ...,  0.0643, -0.0128,  0.0029],
         [-0.2512, -0.0716,  0.2274,  ..., -0.0650, -0.0475, -0.1648],
         [ 0.0293, -0.0548,  0.0157,  ...,  0.0341,  0.0372, -0.0240]],
        device='cuda:0'),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Motif-guided ProtFlamingo

## Helper Functions + Gated Cross Attn + Perceiver Resampler

In [26]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
# from transformers import RobertaModel  # Assuming use of Hugging Face's transformer models

# Helper Functions
def exists(val):
    return val is not None

def set_module_requires_grad_(module, requires_grad):
    for param in module.parameters():
        param.requires_grad = requires_grad

def freeze_model_and_make_eval_(model):
    model.eval()
    set_module_requires_grad_(model, False)

# LayerNorm class
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.gain = nn.Parameter(torch.ones(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.gain * (x - mean) / (std + self.eps)

# Residual class
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, *args, **kwargs):
        return self.fn(x, *args, **kwargs) + x

# SwiGLU activation function
class SwiGLU(nn.Module):
    def forward(self, x):
        return F.silu(x[..., :x.shape[-1] // 2]) * x[..., x.shape[-1] // 2:]

# Transformer Block class
class TransformerBlock(nn.Module):
    def __init__(self, dim, heads, mlp_dim):
        super().__init__()
        self.ln1 = LayerNorm(dim)
        self.attn = nn.MultiheadAttention(dim, heads)
        self.ln2 = LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            SwiGLU(),
            nn.Linear(mlp_dim // 2, dim)
        )
        self.residual = Residual(self.ln1)
        self.feedforward = Residual(self.ln2)
        self.expand_dim = nn.Linear(dim, 2 * dim)  # Project to a higher dimension

    def forward(self, x):
        if x.dim() < 3: ### do the 1,1,1024 transformation
            # Apply the expansion transformation if x has less than 3 dimensions
            x_expanded = self.expand_dim(x)  # Now [2, 2*desired_dim]
            x_expanded = x_expanded.view(1, 1, 1024)  # Reshape to [1, 1, 1024]
            # x_expanded = nn.LayerNorm(x)
            print('x transformed shape in gated cross attn:', x_expanded.shape)
            x = self.residual(self.attn(x_expanded, x_expanded, x_expanded)[0])
        else:
            x = self.residual(self.attn(x, x, x)[0])
        print("Shape after attention and residual:", x.shape)  # Debug print
        x = self.feedforward(self.mlp(x))
        print("Shape after feedforward:", x.shape)  # Debug print
        return x


In [27]:
!pip install transformers

In [28]:
!pip install einops-exts

In [29]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:4096'

import torch
from torch import nn, einsum
import torch.nn.functional as F

from einops import rearrange, repeat
from einops_exts import rearrange_many, repeat_many

def exists(val):
    return val is not None

def FeedForward(dim, mult = 4):
    inner_dim = int(dim * mult)
    return nn.Sequential(
        nn.LayerNorm(dim),
        nn.Linear(dim, inner_dim, bias = False),
        nn.GELU(),
        nn.Linear(inner_dim, dim, bias = False)
    )

class PerceiverAttention(nn.Module):
    def __init__(self, *, dim, concatenated_dim, dim_head=64, heads=8):
        super().__init__()
        self.scale = dim_head ** -0.5
        self.heads = heads
        inner_dim = dim_head * heads

        self.norm_media = nn.LayerNorm(dim)
        self.norm_latents = nn.LayerNorm(dim)
        self.to_q = nn.Linear(dim, inner_dim, bias=False)
        self.to_kv = nn.Linear(dim, inner_dim * 2, bias=False)
        self.to_out = nn.Linear(inner_dim, dim, bias=False)

    def forward(self, x, latents):
        x = self.norm_media(x)
        latents = self.norm_latents(latents)

        b, _, _ = x.shape
        if latents.dim() == 4:
            b, _, m, _ = latents.shape

            q = self.to_q(latents)
            kv_input = torch.cat((x.unsqueeze(2), latents), dim=2)
            k, v = self.to_kv(kv_input).chunk(2, dim=-1)

            q, k, v = rearrange_many((q, k, v), 'b t n (h d) -> b h t n d', h=self.heads)
            q = q * self.scale

            sim = einsum('... i d, ... j d -> ... i j', q, k)
            attn = sim.softmax(dim=-1)
            out = einsum('... i j, ... j d -> ... i d', attn, v)
            out = rearrange(out, 'b h t n d -> b t n (h d)', h=self.heads)
            return self.to_out(out)
        else:
            raise ValueError("Unexpected latents dimension")

class PerceiverResampler(nn.Module):
    def __init__(self, *, dim, depth, dim_head=64, heads=8, num_latents=64, concatenated_dim=2048):
        super().__init__()
        self.latents = nn.Parameter(torch.randn(num_latents, dim))
        self.layers = nn.ModuleList([])

        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PerceiverAttention(dim=dim, concatenated_dim=concatenated_dim, dim_head=dim_head, heads=heads),
                FeedForward(dim=dim)
            ]))

    def forward(self, x):
        latents = repeat(self.latents, 'n d -> b n d', b=x.shape[0])

        for attn, ff in self.layers:
            latents = attn(x, latents) + latents
            latents = ff(latents) + latents

        return latents

class MaskedCrossAttention(nn.Module):
    def __init__(self, *, dim, concatenated_dim=2048, dim_head=64, heads=8, only_attend_immediate_media=True):
        super().__init__()
        self.scale = dim_head ** -0.5
        self.heads = heads
        inner_dim = dim_head * heads

        self.norm = nn.LayerNorm(dim)
        self.to_q = nn.Linear(dim, inner_dim, bias=False)
        self.to_kv = nn.Linear(dim, inner_dim * 2, bias=False)
        self.to_out = nn.Linear(inner_dim, dim, bias=False)
        self.only_attend_immediate_media = only_attend_immediate_media

    def forward(self, x, media, media_locations=None):
        b, t, m = media.shape[:3]
        h = self.heads

        x = self.norm(x)
        q = self.to_q(x)
        media = rearrange(media, 'b t n d -> b (t n) d')
        k, v = self.to_kv(media).chunk(2, dim=-1)
        q, k, v = rearrange_many((q, k, v), 'b n (h d) -> b h n d', h=h)

        q = q * self.scale
        sim = einsum('... i d, ... j d -> ... i j', q, k)
        if media_locations is not None:
            mask = media_locations.unsqueeze(1).unsqueeze(2)
            sim = sim.masked_fill(mask == 0, float('-inf'))

        sim = sim - sim.amax(dim=-1, keepdim=True).detach()
        attn = sim.softmax(dim=-1)
        out = einsum('... i j, ... j d -> ... i d', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)


class GatedCrossAttentionBlock(nn.Module):
    def __init__(self, *, dim, dim_head=64, heads=8, ff_mult=4, only_attend_immediate_media=True):
        super().__init__()
        self.attn = MaskedCrossAttention(dim=dim, concatenated_dim=2048, dim_head=dim_head, heads=heads, only_attend_immediate_media=only_attend_immediate_media)
        self.attn_gate = nn.Parameter(torch.tensor([0.]))
        self.ff = FeedForward(dim, mult=ff_mult)
        self.ff_gate = nn.Parameter(torch.tensor([0.]))

    def forward(self, x, media, media_locations=None):
        gate = self.attn_gate.tanh()
        x = self.attn(x, media, media_locations=media_locations) * gate + x
        x = self.ff(x) * self.ff_gate.tanh() + x
        return x



## ProtFlamingo

In [30]:

class LayerNorm(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.norm = nn.LayerNorm(dim)

    def forward(self, x):
        return self.norm(x)

class SwiGLU(nn.Module):
    def forward(self, x):
        x, gate = x.chunk(2, dim=-1)
        return F.silu(gate) * x

class ParallelTransformerBlock(nn.Module):
    def __init__(self, dim, dim_head=64, heads=8, ff_mult=4):
        super().__init__()
        self.norm = LayerNorm(dim)

        attn_inner_dim = dim_head * heads
        ff_inner_dim = dim * ff_mult

        self.attn = nn.MultiheadAttention(embed_dim=dim, num_heads=heads)
        self.ff = nn.Sequential(
            nn.Linear(dim, 2* ff_mult * dim),
            SwiGLU(),
            nn.Linear(ff_mult * dim, dim)
        )

    def forward(self, x):
        print("Input to ParallelTransformerBlock:", x.shape)

        x = self.norm(x)
        print("After LayerNorm:", x.shape)

        x = x.permute(1, 0, 2)  # Rearrange for nn.MultiheadAttention
        print("After permute for MultiheadAttention:", x.shape)

        attn_output, _ = self.attn(x, x, x)
        print("After MultiheadAttention:", attn_output.shape)

        x = attn_output + x
        print("After adding attn_output:", x.shape)

        x = x.permute(1, 0, 2)  # Rearrange back
        print("After permute back:", x.shape)

        # ff_output = self.ff(x)
        # print("After FeedForward:", ff_output.shape)
        ff_output = x
        for layer in self.ff:
            if isinstance(layer, nn.Linear):
                print("Input to Linear Layer:", ff_output.shape)
                ff_output = layer(ff_output)
                print("Output from Linear Layer:", ff_output.shape)
            else:
                # Assuming SwiGLU or other non-linear layers don't change shape
                ff_output = layer(ff_output)

        output = ff_output + x
        print("Output from ParallelTransformerBlock:", output.shape)

        return output



In [31]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
import re

In [32]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from transformers import T5ForConditionalGeneration, T5Tokenizer
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer


In [33]:
T5ForConditionalGeneration.from_pretrained("Rostlab/prot_t5_xl_bfd").config.d_model

1024

In [34]:
# Load ProtT5 model
protT5_model = T5ForConditionalGeneration.from_pretrained("Rostlab/prot_t5_xl_bfd")
protT5_tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_bfd")

In [35]:
import torch
from torch import nn
from transformers import T5ForConditionalGeneration, T5Tokenizer

class ProtFlamingo(nn.Module):
    def __init__(self, num_tokens, depth, dim_head=64, heads=8, ff_mult=4, cross_attn_every=3, perceiver_num_latents=64, perceiver_depth=2, motif_mode=False):
        super().__init__()

        self.dim = 1024  # Assuming the embedding dimension
        self.to_logits = nn.Linear(self.dim, num_tokens)

        self.perceiver_resampler = PerceiverResampler(dim=self.dim, depth=perceiver_depth, dim_head=dim_head, heads=heads, num_latents=perceiver_num_latents)

        self.layers = nn.ModuleList([])
        for i in range(depth):
            # Note that we're no longer passing 'mlp_dim'
            self.layers.append(ParallelTransformerBlock(dim=self.dim, dim_head=dim_head, heads=heads, ff_mult=ff_mult))
            if i % cross_attn_every == 0:
                self.layers.append(GatedCrossAttentionBlock(dim=self.dim, dim_head=dim_head, heads=heads))


    def forward(self, protein_embeddings, motif_encodings, target_sequence):

        # Interleave protein and motif embeddings
        combined_embeddings = self.interleave_embeddings(protein_embeddings, motif_encodings)

        # Process protein embeddings through the perceiver resampler
        processed_protein_embeddings = self.perceiver_resampler(protein_embeddings)

        for index, layer in enumerate(self.layers):
            if isinstance(layer, GatedCrossAttentionBlock):
                # Pass motif_encodings as media
                target_sequence = layer(target_sequence, motif_encodings)
            else:
                target_sequence = layer(target_sequence)

        logits = self.to_logits(target_sequence)
        return logits

    def interleave_embeddings(self, protein_embeddings, motif_embeddings):
        # Check for matching batch sizes and sequence lengths
        assert protein_embeddings.size(0) == motif_embeddings.size(0), "Mismatch in batch sizes"
        assert protein_embeddings.size(1) == motif_embeddings.size(1), "Mismatch in sequence lengths"

        # Interleave embeddings
        batch_size, seq_length, emb_dim = protein_embeddings.size()
        combined_embeddings = torch.zeros(batch_size, seq_length * 2, emb_dim, device=protein_embeddings.device)

        combined_embeddings[:, 0::2, :] = protein_embeddings
        combined_embeddings[:, 1::2, :] = motif_embeddings

        return combined_embeddings

## Train Model

In [36]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    seq1_embeddings, seq2_embeddings, one_hot_scores, tokenized_seqs = zip(*batch)

    seq1_embeddings = pad_sequence(seq1_embeddings, batch_first=True)
    seq2_embeddings = pad_sequence(seq2_embeddings, batch_first=True)
    one_hot_scores = pad_sequence(one_hot_scores, batch_first=True)
    tokenized_seqs = pad_sequence(tokenized_seqs, batch_first=True, padding_value=tokenizer.pad_token_id)

    return seq1_embeddings, seq2_embeddings, one_hot_scores, tokenized_seqs


In [37]:
import torch
import torch.nn as nn
from torch.nn.functional import kl_div, log_softmax


In [38]:
import torch

# Assuming 'model', 'train_dataloader', 'val_dataloader', 'test_dataloader', and 'criterion' are already defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model = model.half() if device.type == 'cuda' else model.full()


optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)



In [39]:
train_dataloader.dataset

In [40]:
# Instantiate model, optimizer, and other training components
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Example parameters
num_tokens = protT5_tokenizer.vocab_size
depth = 12  # Adjust based on model complexity and computational resources

In [41]:
model = ProtFlamingo(
    num_tokens=num_tokens,
    depth=depth,
    dim_head=64,
    heads=8,
    ff_mult=4,
    cross_attn_every=2,
    perceiver_num_latents=64,
    perceiver_depth=2
).to(device)


In [53]:
def train_epoch_kl(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0

    for seq1_embeddings, seq2_embeddings, one_hot_scores, tokenized_seqs in data_loader:
        # seq1_embeddings, seq2_embeddings, one_hot_scores, tokenized_seqs = \
        #     # Convert all tensors to FloatTensor
        seq1_embeddings = seq1_embeddings.float().to(device)
        print(seq1_embeddings.shape)
        seq2_embeddings = seq2_embeddings.float().to(device)
        print(seq2_embeddings.shape)
        one_hot_scores = one_hot_scores.float().to(device)
        print(one_hot_scores.shape)
        tokenized_seqs = tokenized_seqs.float().to(device)  # Convert tokenized_seqs to float
        print(tokenized_seqs.shape)


        optimizer.zero_grad()

        model_output = model(seq1_embeddings, seq2_embeddings, one_hot_scores)
        print('model output shape:', model_output.shape)
        log_probs = F.log_softmax(model_output, dim=-1)
        print('log probs shape:', log_probs.shape)
        print('tokenized_seqs shape:', tokenized_seqs.shape)
        print(tokenized_seqs)
        print(log_probs)

        loss = F.kl_div(log_probs, tokenized_seqs, reduction='batchmean')
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(data_loader)

# Train for one epoch
train_loss = train_epoch_kl(model, train_dataloader, optimizer, device)
print(f"Training Epoch: Loss = {train_loss}")

def validate_epoch_kl(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for seq1_embeddings, seq2_embeddings, one_hot_scores, tokenized_seqs in data_loader:
            # seq1_embeddings, seq2_embeddings, one_hot_scores, tokenized_seqs = \
            #     # Convert all tensors to FloatTensor
            seq1_embeddings = seq1_embeddings.float().to(device)
            seq2_embeddings = seq2_embeddings.float().to(device)
            one_hot_scores = one_hot_scores.float().to(device)
            tokenized_seqs = tokenized_seqs.float().to(device)  # Convert tokenized_seqs to float

            model_output = model(seq1_embeddings, seq2_embeddings, one_hot_scores)
            log_probs = F.log_softmax(model_output, dim=-1)

            loss = F.kl_div(log_probs, tokenized_seqs, reduction='batchmean')
            total_loss += loss.item()

    return total_loss / len(data_loader)

# Validate for one epoch
val_loss = validate_epoch_kl(model, val_dataloader, device)
print(f"Validation Epoch: Loss = {val_loss}")


def test_epoch_kl(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for seq1_embeddings, seq2_embeddings, one_hot_scores, tokenized_seqs in data_loader:
            # seq1_embeddings, seq2_embeddings, one_hot_scores, tokenized_seqs = \
            #     # Convert all tensors to FloatTensor
            seq1_embeddings = seq1_embeddings.float().to(device)
            seq2_embeddings = seq2_embeddings.float().to(device)
            one_hot_scores = one_hot_scores.float().to(device)
            tokenized_seqs = tokenized_seqs.float().to(device)  # Convert tokenized_seqs to float

            model_output = model(seq1_embeddings, seq2_embeddings, one_hot_scores)
            log_probs = F.log_softmax(model_output, dim=-1)

            loss = F.kl_div(log_probs, tokenized_seqs, reduction='batchmean')
            total_loss += loss.item()

    return total_loss / len(data_loader)

# Test for one epoch
test_loss = test_epoch_kl(model, test_dataloader, device)
print(f"Test Epoch: Loss = {test_loss}")


torch.Size([2, 344, 1024])
torch.Size([2, 344, 1024])
torch.Size([2, 344])
torch.Size([2, 344])


ValueError: ignored

In [ ]:
protT5_tokenizer